## Importando Bibliotecas

In [75]:
import tensorflow as tf 
from tensorflow import keras
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Input, Dense
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from keras.optimizers import Adam
from keras.models import Model
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler

## Pre-processamento dos dados
**Carregando os dados de treino e teste:**

In [76]:
train_images = pd.read_csv('../fashion-mnist_train.csv')
test_images = pd.read_csv('../fashion-mnist_test.csv')

**Separando as labels do conjuntos:**

In [77]:
train_labels=train_images.loc[:, ['label']]
train_images=train_images.drop(['label'], axis=1)
test_labels=test_images.loc[:, ['label']]
test_images=test_images.drop(['label'], axis=1)

**Normalizando os data-sets:**

In [78]:
train_images = train_images / 255.0
test_images = test_images / 255.0


----------------------
## Construção do modelo base
**Esta rede será utilizada posteriomente para comparações entre os modelos de dimensionalidade reduzida utilizando PCA e autoencoder.**

**Definição da estrutura do modelo por meio do Keras Sequential:**

In [79]:
model = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(784)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_16 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1290      
Total params: 59,850
Trainable params: 59,850
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [80]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [81]:
model.fit(train_images, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 7s 150us/sample - loss: 0.5294 - accuracy: 0.8114 - val_loss: 0.4116 - val_accuracy: 0.8508
Epoch 2/5
48000/48000 [==============================] - 6s 123us/sample - loss: 0.3894 - accuracy: 0.8590 - val_loss: 0.3898 - val_accuracy: 0.8615
Epoch 3/5
48000/48000 [==============================] - 5s 106us/sample - loss: 0.3526 - accuracy: 0.8713 - val_loss: 0.3741 - val_accuracy: 0.8697
Epoch 4/5
48000/48000 [==============================] - 6s 115us/sample - loss: 0.3286 - accuracy: 0.8801 - val_loss: 0.3586 - val_accuracy: 0.8685
Epoch 5/5
48000/48000 [==============================] - 6s 126us/sample - loss: 0.3106 - accuracy: 0.8847 - val_loss: 0.3373 - val_accuracy: 0.8818


**Verificação dos resultados do modelo:**

In [82]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 48us/sample - loss: 0.3224 - accuracy: 0.8792
Model - 3 layers - test loss: 32.24343364477157
Model - 3 layers - test accuracy: 87.91999816894531


-----------------------------
## Redução de Dimensionalidade Usando Principal component analysis (PCA)

**Preprocessamento de dados (Feature Scaling)**

In [99]:
scaler = StandardScaler()
scaler.fit(train_images)

train_images_r = scaler.transform(train_images)
test_images_r = scaler.transform(test_images)

**Definição do número de componentes:**

In [98]:
pca = PCA(n_components=3)

**Construção dos novos datasets de treino e de teste com dimensões reduzidas usando o PCA:**

In [92]:
pca.fit(train_images_r)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [93]:
train_images_r = pca.transform(train_images_r)
test_images_r = pca.transform(test_images_r)

**Alteração do tipo dos dados de numpy array para pandas dataframe:**

In [94]:
train_images_r = pd.DataFrame(data=train_images_r)
test_images_r= pd.DataFrame(data=test_images_r)

**Definição da estrutura do modelo por meio do Keras Sequential que receberá os dados com dimensões reduzidas pelo PCA:**

In [95]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(3)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 64)                256       
_________________________________________________________________
dense_19 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_20 (Dense)             (None, 10)                1290      
Total params: 9,866
Trainable params: 9,866
Non-trainable params: 0
_________________________________________________________________


**Compilação do modelo:**

In [96]:
model_r.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**Treino do modelo:**

In [97]:
model_r.fit(train_images_r, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 61us/sample - loss: 1.0286 - accuracy: 0.5805 - val_loss: 0.9906 - val_accuracy: 0.6049
Epoch 2/5
48000/48000 [==============================] - 2s 52us/sample - loss: 0.9523 - accuracy: 0.6087 - val_loss: 0.9385 - val_accuracy: 0.6182
Epoch 3/5
48000/48000 [==============================] - 3s 54us/sample - loss: 0.9247 - accuracy: 0.6235 - val_loss: 0.9220 - val_accuracy: 0.6273
Epoch 4/5
48000/48000 [==============================] - 3s 57us/sample - loss: 0.9043 - accuracy: 0.6320 - val_loss: 0.9173 - val_accuracy: 0.6223
Epoch 5/5
48000/48000 [==============================] - 2s 51us/sample - loss: 0.8945 - accuracy: 0.6391 - val_loss: 0.8988 - val_accuracy: 0.6375


**Verificação dos resultados do modelo:**

In [61]:
test_loss, test_acc = model_r.evaluate(test_images_r, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 29us/sample - loss: 0.8587 - accuracy: 0.6429
Model - 3 layers - test loss: 85.8690246963501
Model - 3 layers - test accuracy: 64.28999900817871


------------------------------------------------------

## Redução de Dimensionalidade Usando Autoencoders

In [16]:
# this is the size of our encoded representations
encoding_dim = 2
input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
encoder = Model(input_img, encoded)
decoded = Dense(784, activation='sigmoid')(encoded)
autoencoder = Model(input_img, decoded)
encoded_input = Input(shape=(encoding_dim,))
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [17]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [18]:
#REMOVER troque o 'adadelta' pelo 'adam'
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
autoencoder.fit(train_images, train_images,epochs=5,shuffle=True,validation_data=(test_images, test_images))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.4380 - val_loss: 0.4080
Epoch 2/5
60000/60000 [==============================] - 6s 105us/step - loss: 0.4010 - val_loss: 0.3940
Epoch 3/5
60000/60000 [==============================] - 6s 103us/step - loss: 0.3904 - val_loss: 0.3854
Epoch 4/5
60000/60000 [==============================] - 6s 105us/step - loss: 0.3832 - val_loss: 0.3794
Epoch 5/5
60000/60000 [==============================] - 6s 107us/step - loss: 0.3789 - val_loss: 0.3768


In [20]:
encoded_imgs_train = encoder.predict(train_images)
encoded_imgs_test = encoder.predict(test_images)


In [21]:
train_images_a = pd.DataFrame(data=encoded_imgs_train)
test_images_a =pd.DataFrame(data=encoded_imgs_test)

In [22]:
model_r = keras.Sequential([
    keras.layers.Dense(64,activation=tf.nn.relu,input_dim=(2)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model_r.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 9,802
Trainable params: 9,802
Non-trainable params: 0
_________________________________________________________________


In [23]:
model_r.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [24]:
model_r.fit(train_images_a, train_labels, epochs=5, validation_split=0.2, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 3s 68us/sample - loss: 1.2822 - accuracy: 0.4715 - val_loss: 1.1831 - val_accuracy: 0.5287
Epoch 2/5
48000/48000 [==============================] - 3s 66us/sample - loss: 1.1723 - accuracy: 0.5204 - val_loss: 1.1552 - val_accuracy: 0.5290- loss: 1.1758  - ETA: 0s -
Epoch 3/5
48000/48000 [==============================] - 3s 54us/sample - loss: 1.1512 - accuracy: 0.5291 - val_loss: 1.1430 - val_accuracy: 0.5357
Epoch 4/5
48000/48000 [==============================] - 3s 62us/sample - loss: 1.1437 - accuracy: 0.5310 - val_loss: 1.1623 - val_accuracy: 0.5228
Epoch 5/5
48000/48000 [==============================] - 3s 56us/sample - loss: 1.1379 - accuracy: 0.5351 - val_loss: 1.1354 - val_accuracy: 0.5397


In [25]:
test_loss, test_acc = model_r.evaluate(test_images_a, test_labels)
print("Model - 3 layers - test loss:", test_loss * 100)
print("Model - 3 layers - test accuracy:", test_acc * 100)

10000/10000 [==============================] - 0s 35us/sample - loss: 1.1422 - accuracy: 0.5352
Model - 3 layers - test loss: 114.21793071746826
Model - 3 layers - test accuracy: 53.519999980926514


------------------------------------------------------

## Clustering com K-means

Optamos pela utilização das imagens reduzidas por meio do PCA, já que obtiveram por meio deste modelo os melhores resultados de predição na rede neural.

In [26]:
train_images_k = train_images_r
test_images_k = test_images_r

Construção do modelo do k-means com 10 clusters.

In [27]:
kmean_model = KMeans(n_clusters=10, random_state=10)
y_km = kmean_model.fit(train_images_k)
labels = kmean_model.labels_

In [28]:
cluster_centroids = kmean_model.cluster_centers_

In [29]:
#silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [30]:
silhouette_avg = silhouette_score(X = train_images_k, labels = train_labels.values.ravel(), random_state=10)

In [33]:
#metrics.homogeneity_score(train_labels.values.ravel(), labels)

In [34]:
metrics.v_measure_score(train_labels.values.ravel(), labels, beta=1.0)

0.45694727123110823

---------------------------------
## Clustering com o DBScan
